In [40]:
import os
import numpy as np
import scipy
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Dense, Flatten, Conv2D, Lambda, Input, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, Callback
from tensorflow.summary import create_file_writer

# Set up Configuration 

In [41]:
# Model Configuration
def model_configuration(): 
    """
    Get the model configuration
    """

    # Load dataset for computing dataset size
    (input_train, _), (_, _) = load_dataset()

    # Generic config
    width, height, channels = 32, 32, 3
    batch_size = 128
    num_classes = 10
    validation_split = 0.1
    verbose = 1
    n = 3
    init_fm_dim = 16
    shortcut_type = "identity"
    
    # Dataset size
    train_size = (1 - validation_split)  * len(input_train)
    val_size = (validation_split) * len(input_train)

    # Number of steps per epoch is dependent on batch size
    maximum_number_iterations = 64000
    steps_per_epoch = tensorflow.math.floor(train_size / batch_size)
    val_steps_per_epoch = tensorflow.math.floor(val_size / batch_size)
    epochs = tensorflow.cast(tensorflow.math.floor(maximum_number_iterations / steps_per_epoch), dtype=tensorflow.int64)

    # Define loss function
    loss = tensorflow.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Learning rate config
    boundaries = [32000, 48000]
    values = [0.1, 0.01, 0.001]
    lr_schedule = schedules.PiecewiseConstantDecay(boundaries, values)

    # Set layer init
    initialiser = tensorflow.keras.initializers.HeNormal()

    # Define optimiser
    optimiser_momentum = 0.9
    optimiser_additional_metrics = ["accuracy"]
    optimiser = SGD(learning_rate=lr_schedule, momentum=optimiser_momentum)

    # Load Tensorboard callback
    tensorboard = TensorBoard(
	    log_dir=os.path.join(os.getcwd(), "logs"),
	    histogram_freq=1,
	    write_images=True
	)

    # Save a model checkpoint after every epoch
    checkpoint = ModelCheckpoint(
        os.path.join(os.getcwd(), "model_checkpoint"),
        save_freq="epoch"
    )

    # Early stopping callback
    early_stopping = EarlyStopping(monitor="val_loss", patience=10, verbose=1, restore_best_weights=True)

    # Summary Writer callback
    class SummaryWriter(Callback):
        def __init__(self, log_dir):
            super().__init__()
            self.log_dir = log_dir

        def on_epoch_end(self, epoch, logs=None):
            with create_file_writer(self.log_dir).as_default():
                for key, value in logs.items():
                    tensorflow.summary.scalar(key, value, step=epoch)

    # Add callbacks to list
    callbacks = [
        SummaryWriter(log_dir=os.path.join(os.getcwd(), "logs")),
        tensorboard,
        checkpoint
    ]

    #Create config dictionary
    config = {
        "width": width,
        "height": height,
        "dim": channels,
        "batch_size": batch_size,
        "num_classes": num_classes,
        "validation_split": validation_split,
        "verbose": verbose,
        "stack_n": n,
        "initial_num_feature_maps": init_fm_dim,
        "training_ds_size": train_size,
        "steps_per_epoch": steps_per_epoch,
        "val_steps_per_epoch": val_steps_per_epoch,
        "num_epochs": epochs,
        "loss": loss,
        "optim": optimiser,
        "optim_learning_rate_schedule": lr_schedule,
        "optim_momentum": optimiser_momentum,
        "optim_additional_metrics": optimiser_additional_metrics,
        "initialiser": initialiser,
        "callbacks": callbacks,
        "shortcut_type": shortcut_type,
        "name": "ResNet20",
        "dropout_rate": 0.5
    }
    return config

# Loading Dataset

In [42]:
def load_dataset():
	"""
		Load the CIFAR-10 dataset
	"""
	return cifar10.load_data()

# Preprocessing
## Cropping image

In [43]:
def random_crop(img, random_crop_size): 
    # Note: image_data_format is 'channel_last'
    # SOURCE: https://jkjung-avt.github.io/keras-image-cropping/

    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    """
    Take as input a Keras ImageGen (Iterator) and generate random crops from the image batches generated by the original iterator
    SOURCE: https://jkjung-avt.github.io/keras-image-cropping/
    """

    while True: 
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [44]:
def preprocessed_dataset(): 
    """
    Load and preprocess the CIFAR-10 dataset
    """
    (input_train, target_train), (input_test, target_test) = load_dataset()

    # Retrieve shape from model configuration and unpack into components
    config = model_configuration()
    width, height, dim = config.get("width"), config.get("height"), config.get("dim")
    num_classes = config.get("num_classes")

    # Data augmentation: perform zero padding on datasets
    paddings = tensorflow.constant([[0,0,], [4,4], [4,4], [0,0]])
    input_train = tensorflow.pad(input_train, paddings, mode="CONSTANT")

    # Convert scalar targets to categorical ones
    target_train = tensorflow.keras.utils.to_categorical(target_train, num_classes)
    target_test = tensorflow.keras.utils.to_categorical(target_test, num_classes)

    # Data generator for training data
    train_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        validation_split=config.get("validation_split"),
        horizontal_flip=True,
        rescale=1./255,
        preprocessing_function = tensorflow.keras.applications.resnet50.preprocess_input
    )

    # Generate training and validation batches
    train_batches = train_generator.flow(input_train, target_train, batch_size=config.get("batch_size"), subset="training")
    validation_batches = train_generator.flow(input_train, target_train, batch_size=config.get("batch_size"), subset="validation")
    train_batches = crop_generator(train_batches, config.get("height"))
    validation_batches = crop_generator(validation_batches, config.get("height"))

    # Data generator for testing data
    test_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        preprocessing_function = tensorflow.keras.applications.resnet50.preprocess_input
    )

    # Generate testing batches
    test_batches = test_generator.flow(input_test, target_test, batch_size=config.get("batch_size"))

    return train_batches, validation_batches, test_batches

# Creating the Residual Block

In [45]:
def residual_block(x, number_of_filters, match_filter_size=False):
    """
    Residual block for ResNet
    """
    # Retrieve initialiser
    config = model_configuration()
    initialiser = config.get("initialiser")

    # Create skip connection
    x_skip = x

    # Perform the original mapping
    if match_filter_size: 
        x = Conv2D(number_of_filters, kernel_size=(3, 3), strides=(2, 2), kernel_initializer=initialiser,padding="same")(x_skip)
    else: 
        x = Conv2D(number_of_filters, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initialiser, padding="same")(x_skip)

    x = BatchNormalization(axis=3)(x)
    x = Activation("relu")(x)
    x = Conv2D(number_of_filters, kernel_size=(3, 3), kernel_initializer=initialiser, padding="same")(x)
    x = BatchNormalization(axis=3)(x)

    # Perform matching of filter numbers if necessary
    if match_filter_size and config.get("shortcut_type") == "identity": 
        x_skip = Lambda(lambda x: tensorflow.pad(x[:, ::2, ::2, :], tensorflow.constant([[0, 0,], [0, 0], [0, 0], [number_of_filters//4, number_of_filters//4]]), mode="CONSTANT"))(x_skip)
        x_skip = BatchNormalization(axis=3)(x_skip)
    elif match_filter_size and config.get("shortcut_type") == "projection":
        x_skip = Conv2D(number_of_filters, kernel_size=(1,1), kernel_initializer=initializer, strides=(2,2))(x_skip)
    
    # Add the skip connection to the regular mapping
    x = Add()([x, x_skip])

    # Nonlinearly activate the result
    x = Activation("relu")(x)

    # Return the result
    return x

In [46]:
def ResidualBlocks(x):
    """ 
    Set up the residual blocks
    """

    # Retrieve values
    config = model_configuration()

    # Set initial filter size
    filter_size = config.get("initial_num_feature_maps")

    # 6n/2n = 3, there are 3 groups of residual blocks
    for layer_group in range(3): 

        # Each block in our code has 2 weighted layers, and each group has 2n such blocks, so we have n blocks per group
        for block in range(config.get("stack_n")):

            # Perform filter size increase at every first layer in the 2nd block onwards. 
            if layer_group > 0 and block == 0: 
                filter_size *= 2
                x = residual_block(x, filter_size, match_filter_size=True)
            else:
                x = residual_block(x, filter_size)

    return x

In [47]:
def model_base(shp): 
    """
    Base Structure of the model, with residual blocks attached
    """

    # Get number of classes from model configuration
    config = model_configuration()
    initialiser = config.get("initialiser")

    # Define model structure
    # logits are returned becase Softmax is pushed to loss function
    inputs = Input(shape=shp)
    x = Conv2D(config.get("initial_num_feature_maps"), kernel_size=(3, 3), strides=(1,1), kernel_initializer=initialiser, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = ResidualBlocks(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(512, kernel_initializer=initialiser)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(config.get("dropout_rate"))(x)
    outputs = Dense(config.get("num_classes"), kernel_initializer=initialiser)(x)

    return inputs, outputs

# Model Initialisation

In [48]:
def init_model(): 
    """
    Initilise a compiled ResNet model
    """

    # Get shape from model configuration
    config = model_configuration()

    # Get model base
    inputs, outputs = model_base((config.get("width"), config.get("height"), config.get("dim")))

    # Initialise and compile model
    model = Model(inputs, outputs, name=config.get("name"))
    model.compile(loss=config.get("loss"), optimizer=config.get("optim"), metrics=config.get("optim_additional_metrics"))

    # Print model summary
    model.summary()

    return model

# Training

In [49]:
def train_model(model, train_batches, validation_batches): 
    """
    Train an initialised model
    """

    # Get model configuration
    config = model_configuration()

    # Fit data to model
    model.fit(
        train_batches,
        batch_size=config.get("batch_size"),
        epochs=config.get("num_epochs"),
        verbose=config.get("verbose"),
        callbacks=config.get("callbacks"),
        steps_per_epoch=config.get("steps_per_epoch"),
        validation_data=validation_batches,
        validation_steps=config.get("val_steps_per_epoch")
    )
    return model


# Evaluation

In [50]:
def evaluate_model(model, test_batches): 
    """
    Evaluate a trained model
    """

    # Evaluate the model
    score = model.evaluate(test_batches, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Overall Process

In [51]:
def training_process(): 
    """
    Run the training process for the ResNet model
    """

    # Get dataset
    train_batches, validation_batches, test_batches = preprocessed_dataset()

    # Initialise ResNet
    resnet = init_model()

    # Train ResNet model
    trained_resnet = train_model(resnet, train_batches, validation_batches)

    # Evaluate ResNet model
    evaluate_model(trained_resnet, test_batches)

In [52]:
if __name__ == "__main__": 
    training_process()

/Users/danielhuang/venv-metal/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "ResNet20"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_57 (Conv2D)          (None, 32, 32, 16)           448       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_64 (Ba  (None, 32, 32, 16)           64        ['conv2d_57[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_58 (Activation)  (None, 32, 32, 16)           0         ['batch_normalization_6

INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


351/351 [==============================] - 31s 78ms/step - loss: 2.3198 - accuracy: 0.2478 - val_loss: 1.8528 - val_accuracy: 0.2965
Epoch 2/182
351/351 [==============================] - ETA: 0s - loss: 1.6810 - accuracy: 0.3750INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


...

351/351 [==============================] - 21s 61ms/step - loss: 0.0622 - accuracy: 0.9776 - val_loss: 0.4978 - val_accuracy: 0.8844
Epoch 181/182
351/351 [==============================] - ETA: 0s - loss: 0.0639 - accuracy: 0.9762INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


351/351 [==============================] - 22s 63ms/step - loss: 0.0639 - accuracy: 0.9762 - val_loss: 0.5083 - val_accuracy: 0.8789
Epoch 182/182
350/351 [============================>.] - ETA: 0s - loss: 0.0635 - accuracy: 0.9773INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


INFO:tensorflow:Assets written to: /Users/danielhuang/tinyML/Fine-tuning/model_checkpoint/assets


351/351 [==============================] - 21s 61ms/step - loss: 0.0635 - accuracy: 0.9773 - val_loss: 0.5092 - val_accuracy: 0.8734
Test loss: 0.5847798585891724 / Test accuracy: 0.8770999908447266
